In [52]:
import tensorflow as tf
import tensorflow.compat.v2.feature_column as fc
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
tf.random.set_seed(42)


Using Neural Net

In [53]:
CLASS_NAMES = ['Distance_Driver', 'Control_Driver', 'Putt_&_Approach', 'Mid_Range']

In [54]:
disc_data=pd.read_csv("C:/Users/idansim/Dropbox/Work/Python_tutorials/Disc_Golf/disc-data.csv")

In [55]:
# Replace spaces in column names
disc_data.columns = disc_data.columns.str.replace(' ', '_')
disc_data.columns = disc_data.columns.str.replace('[()%]', '', regex=True)

# Replace spaces in string values
disc_data = disc_data.apply(lambda col: col.str.replace(' ', '_') if col.dtype == 'object' else col)

disc_data = disc_data.drop('MOLD', axis=1)

In [56]:
disc_data['BEAD'] = disc_data['BEAD'].replace({'Yes': 1, 'No': 0})

C:\Users\idansim\AppData\Local\Temp\ipykernel_19836\3057763355.py:1: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [57]:
disc_data

,DISC_TYPE,SPEED,GLIDE,TURN,FADE,STABILITY,DIAMETER_cm,HEIGHT_cm,RIM_DEPTH_cm,RIM_WIDTH_cm,INSIDE_RIM_DIAMETER_cm,RIM_DEPTH_/_DIAMETER_RATION_,RIM_CONFIGURATION,BEAD
0,Putt_&_Approach,1.0,2.0,0.0,1.0,1.0,21.3,1.9,1.4,0.8,19.6,6.6,90.75,0
1,Putt_&_Approach,1.0,7.0,-1.0,0.0,-1.0,21.9,2.2,1.6,0.8,20.3,7.3,89.25,0
2,Putt_&_Approach,1.0,1.0,0.0,2.0,2.0,21.1,2.0,1.4,0.9,19.3,6.6,87.00,0
3,Putt_&_Approach,1.0,2.0,0.0,0.0,0.0,21.2,2.4,1.9,0.7,19.8,9.0,98.25,0
4,Putt_&_Approach,1.0,3.0,0.0,0.5,0.5,21.2,2.0,1.7,0.9,19.4,8.0,85.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170,Distance_Driver,14.5,5.0,-0.5,3.0,2.5,21.1,1.8,1.2,2.5,16.0,5.7,27.00,0
1171,Distance_Driver,14.5,5.0,0.0,3.0,3.0,21.0,1.8,1.2,2.5,16.0,5.7,27.75,0
1172,Distance_Driver,14.5,5.5,-2.0,2.0,0.0,21.1,2.1,1.2,2.5,15.9,5.7,27.00,0
1173,Distance_Driver,14.5,3.5,0.0,4.0,4.0,21.1,1.8,1.2,2.5,16.0,5.7,27.25,0


In [58]:
disc_data['BEAD'] = disc_data['BEAD'].replace({'Yes': 1, 'No': 0})

scaler = MinMaxScaler()
numerical_columns = disc_data.select_dtypes(include=['number']).columns
disc_data[numerical_columns] = scaler.fit_transform(disc_data[numerical_columns])



# Initialize LabelEncoder
encoder = LabelEncoder()
# Apply label encoding
disc_data['DISC_TYPE'] = encoder.fit_transform(disc_data['DISC_TYPE'])


In [59]:
# Sample a tenth of the data to use as test data
disc_test = disc_data.sample(n=(disc_data.shape[0]//10), random_state=42)
disc_train = disc_data.drop(disc_test.index)
disc_train.reset_index(drop=True, inplace=True)
disc_test.reset_index(drop=True, inplace=True)

disc_train_type = disc_train.pop('DISC_TYPE')
disc_test_type = disc_test.pop('DISC_TYPE')

assert disc_test.shape[0] == disc_data.shape[0]//10
assert disc_train.shape[0] == disc_data.shape[0]-disc_test.shape[0]

disc_train

,SPEED,GLIDE,TURN,FADE,STABILITY,DIAMETER_cm,HEIGHT_cm,RIM_DEPTH_cm,RIM_WIDTH_cm,INSIDE_RIM_DIAMETER_cm,RIM_DEPTH_/_DIAMETER_RATION_,RIM_CONFIGURATION,BEAD
0,0.0,0.285714,0.714286,0.166667,0.454545,0.09375,0.3750,0.250000,0.10,0.536232,0.340426,0.896194,0.0
1,0.0,1.000000,0.571429,0.000000,0.272727,0.28125,0.5625,0.416667,0.10,0.637681,0.489362,0.875433,0.0
2,0.0,0.142857,0.714286,0.333333,0.545455,0.03125,0.4375,0.250000,0.15,0.492754,0.340426,0.844291,0.0
3,0.0,0.285714,0.714286,0.000000,0.363636,0.06250,0.6875,0.666667,0.05,0.565217,0.851064,1.000000,0.0
4,0.0,0.428571,0.714286,0.083333,0.409091,0.06250,0.4375,0.500000,0.15,0.507246,0.638298,0.816609,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053,1.0,0.714286,0.642857,0.500000,0.590909,0.03125,0.3125,0.083333,0.95,0.014493,0.148936,0.013841,0.0
1054,1.0,0.714286,0.714286,0.500000,0.636364,0.00000,0.3125,0.083333,0.95,0.014493,0.148936,0.024221,0.0
1055,1.0,0.785714,0.428571,0.333333,0.363636,0.03125,0.5000,0.083333,0.95,0.000000,0.148936,0.013841,0.0
1056,1.0,0.500000,0.714286,0.666667,0.727273,0.03125,0.3125,0.083333,0.95,0.014493,0.148936,0.017301,0.0


In [60]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)


In [61]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in disc_train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SPEED', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='GLIDE', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='TURN', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='FADE', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='STABILITY', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='DIAMETER_cm', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='HEIGHT_cm', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='RIM_DEPTH_cm', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='RIM_WIDTH_cm', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='INSIDE_RIM_DIAMETER_cm', shape=(1,), default_value=None, dtype=tf.float3

In [62]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    # hidden_units=[30, 10],
    hidden_units=[30, 30, 10, 10],

    # The model must choose between 3 classes.
    n_classes=4)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\idansim\\AppData\\Local\\Temp\\tmpgo99hn34', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [63]:
classifier.train(
    input_fn=lambda: input_fn(disc_train, disc_train_type, training=True),
    steps=8000)
# We include a lambda to avoid creating an inner function previously

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\idansim\AppData\Local\Temp\tmpgo99hn34\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 1.4056456, step = 0
INFO:tensorflow:global_step/sec: 410.066
INFO:tensorflow:loss = 1.3701565, step = 100 (0.246 sec)
INFO:tensorflow:global_step/sec: 528.954
INFO:tensorflow:loss = 1.3411089, step = 200 (0.188 sec)
INFO:tensorflow:global_step/sec: 534.641
INFO:tensorflow:loss = 1.3210397, step = 300 (0.187 sec)
INFO:tensorflow:global_step/sec: 498.547
INFO:tensorflow:loss = 1.2998707, step = 400 (0.201 sec)
INFO:tensorflow:global_step/sec: 640.882
INFO:tensorflow:loss = 1.2670082, step = 500 (0.156 sec)
INFO:tens

In [64]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(disc_test, disc_test_type, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2024-07-30T23:10:41
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\idansim\AppData\Local\Temp\tmpgo99hn34\model.ckpt-8000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.18204s
INFO:tensorflow:Finished evaluation at 2024-07-30-23:10:41
INFO:tensorflow:Saving dict for global step 8000: accuracy = 0.82905984, average_loss = 0.6075292, global_step = 8000, loss = 0.6075292
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 8000: C:\Users\idansim\AppData\Local\Temp\tmpgo99hn34\model.ckpt-8000

Test set accuracy: 0.829



Using RandomForestRegressor

In [65]:
from sklearn.model_selection import train_test_split

In [66]:
X_train, X_test, y_train, y_test = train_test_split(disc_train, disc_train_type, test_size=0.2, random_state= 42)

In [67]:
from sklearn.ensemble import RandomForestRegressor

In [68]:
rfr = RandomForestRegressor(random_state=42)

In [69]:
rfr.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [70]:
y_pred = rfr.predict(X_test)

In [71]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [72]:
mean_absolute_error(y_test, y_pred)

0.12268867924528301

In [73]:
mean_squared_error(y_test, y_pred)

0.0945240566037736

In [74]:
r2_score(y_test, y_pred)

0.9255038981535062

In [75]:
import optuna

In [76]:
from sklearn.model_selection import cross_val_score

In [77]:
def objective (trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', 10, 50)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 32)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 32)

    model= RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)

    score= cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error', n_jobs= (-1)).mean()

    return score

In [78]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler(seed=42))

[I 2024-07-30 23:10:42,077] A new study created in memory with name: no-name-8bcbda4a-d5ac-412f-b5b4-bd9ce5462e2e


In [79]:
study.optimize(objective, n_trials=200)

[I 2024-07-30 23:10:44,428] Trial 0 finished with value: -0.08866610522384985 and parameters: {'n_estimators': 437, 'max_depth': 48, 'min_samples_split': 24, 'min_samples_leaf': 20}. Best is trial 0 with value: -0.08866610522384985.
[I 2024-07-30 23:10:45,710] Trial 1 finished with value: -0.09265716483159438 and parameters: {'n_estimators': 240, 'max_depth': 16, 'min_samples_split': 3, 'min_samples_leaf': 28}. Best is trial 0 with value: -0.08866610522384985.
[I 2024-07-30 23:10:47,528] Trial 2 finished with value: -0.0951360731400551 and parameters: {'n_estimators': 641, 'max_depth': 39, 'min_samples_split': 2, 'min_samples_leaf': 32}. Best is trial 0 with value: -0.08866610522384985.
[I 2024-07-30 23:10:49,564] Trial 3 finished with value: -0.07684681673009439 and parameters: {'n_estimators': 850, 'max_depth': 18, 'min_samples_split': 7, 'min_samples_leaf': 6}. Best is trial 3 with value: -0.07684681673009439.
[I 2024-07-30 23:10:50,134] Trial 4 finished with value: -0.0838359264531

In [80]:
study.best_params

{'n_estimators': 358,
 'max_depth': 34,
 'min_samples_split': 2,
 'min_samples_leaf': 2}

In [81]:
best_params = study.best_params

In [82]:
import plotly


In [83]:
import sys
print(sys.executable)

c:\Users\idansim\AppData\Local\Programs\Python\Python310\python.exe


In [84]:
import optuna.visualization as vis
vis.plot_optimization_history(study)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'mode': 'markers',
              'name': 'Objective Value',
              'type': 'scatter',
              'x': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                    18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
                    34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
                    50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65,
                    66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81,
                    82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97,
                    98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110,
                    111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122,
                    123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134,
                    135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146,
                    147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158,
                    159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
                    171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182,
                    183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194,
                    195, 196, 197, 198, 199],
              'y': [-0.08866610522384985, -0.09265716483159438,
                    -0.0951360731400551, -0.07684681673009439,
                    -0.08383592645315964, -0.08410220744539584,
                    -0.08624530077090894, -0.07952441059356559,
                    -0.09307516554099445, -0.0843572990182119,
                    -0.09400257350281849, -0.08664728232580649,
                    -0.09201507471538543, -0.09385707721191952, -0.083859904542404,
                    -0.08462985341895397, -0.0924843161605656,
                    -0.08094153797990486, -0.0923891037193952,
                    -0.07682257333986389, -0.07611807525351619,
                    -0.08959778021621949, -0.09112620315278282,
                    -0.0856184071355666, -0.07348143277805834,
                    -0.08671761482190704, -0.09203144049301994,
                    -0.07649660912460728, -0.09299267001269662,
                    -0.08654667483142449, -0.07581579019894702,
                    -0.09281137807138395, -0.08149247464695322,
                    -0.08376277045946837, -0.0947306332326459,
                    -0.08330850169717026, -0.08626191193363211,
                    -0.0828132963625177, -0.08273883912450683,
                    -0.09194194233630103, -0.08656812465737966,
                    -0.07637223795345739, -0.08006736617937596,
                    -0.08006598358985587, -0.07914625132695934,
                    -0.09368664277886932, -0.0865994842544214,
                    -0.09319806334255917, -0.08382084316432639,
                    -0.09175493118184035, -0.09323764979240415,
                    -0.09079163729214064, -0.09141268852491094,
                    -0.08148021598818782, -0.09264313209495478,
                    -0.08402696896221087, -0.08458304910638317,
                    -0.08615297863489838, -0.09194560870848345,
                    -0.07966205636739844, -0.08457795491860555,
                    -0.09501593049937154, -0.0838908726121492,
                    -0.08663634013641182, -0.07822577218274301,
                    -0.08641853371343161, -0.0913562560543966,
                    -0.09251341202098598, -0.0860883936564768,
                    -0.09242646943103247, -0.07664072929448075,
                    -0.08700322984086863, -0.07052907041190361,
                    -0.08655378197728471, -0.07843125560916218,
                    -0.08984203040574157, -0.08368426312177844,
                    -0.07725805932640221, -0.09101419506131218,
                    -0.08253329450721868, -0.08287498224629122,
                    -0.08890194661988404, -0.09220209885007302,
                    -0.07081826524197508, -0.077447646455245

In [85]:
optuna.visualization.plot_parallel_coordinate(study)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'dimensions': [{'label': 'Objective Value',
                              'range': [-0.09548042928446096,
                                        -0.07052907041190361],
                              'values': [-0.08866610522384985,
                                         -0.09265716483159438, -0.0951360731400551,
                                         -0.07684681673009439,
                                         -0.08383592645315964,
                                         -0.08410220744539584,
                                         -0.08624530077090894,
                                         -0.07952441059356559,
                                         -0.09307516554099445, -0.0843572990182119,
                                         -0.09400257350281849,
                                         -0.08664728232580649,
                                         -0.09201507471538543,
                                         -0.09385707721191952, -0.083859904542404,
                                         -0.08462985341895397, -0.0924843161605656,
                                         -0.08094153797990486, -0.0923891037193952,
                                         -0.07682257333986389,
                                         -0.07611807525351619,
                                         -0.08959778021621949,
                                         -0.09112620315278282, -0.0856184071355666,
                                         -0.07348143277805834,
                                         -0.08671761482190704,
                                         -0.09203144049301994,
                                         -0.07649660912460728,
                                         -0.09299267001269662,
                                         -0.08654667483142449,
                                         -0.07581579019894702,
                                         -0.09281137807138395,
                                         -0.08149247464695322,
                                         -0.08376277045946837, -0.0947306332326459,
                                         -0.08330850169717026,
                                         -0.08626191193363211, -0.0828132963625177,
                                         -0.08273883912450683,
                                         -0.09194194233630103,
                                         -0.08656812465737966,
                                         -0.07637223795345739,
                                         -0.08006736617937596,
                                         -0.08006598358985587,
                                         -0.07914625132695934,
                                         -0.09368664277886932, -0.0865994842544214,
                                         -0.09319806334255917,
                                         -0.08382084316432639,
                                         -0.09175493118184035,
                                         -0.09323764979240415,
                                         -0.09079163729214064,
                                         -0.09141268852491094,
                                         -0.08148021598818782,
                                         -0.09264313209495478,
                                         -0.08402696896221087,
                                         -0.08458304910638317,
                                         -0.08615297863489838,
                                         -0.09194560870848345,
                                         -0.07966205636739844,
                                         -0.08457795491860555,
                                         -0.09501593049937154, -0.0838908726121492,
                                         -0.08663634013641182,
                                         -0.07822577218274301,
                                         -0.08641853371343161, -0.0913562560543966,
                                         -0.092513

In [86]:
optuna.visualization.plot_slice(study, params=['n_estimators', 'max_depth', 'min_samples_leaf', 'min_samples_split'])

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                   14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25,
                                   26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37,
                                   38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
                                   50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61,
                                   62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73,
                                   74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
                                   86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97,
                                   98, 99, 100, 101, 102, 103, 104, 105, 106, 107,
                                   108, 109, 110, 111, 112, 113, 114, 115, 116,
                                   117, 118, 119, 120, 121, 122, 123, 124, 125,
                                   126, 127, 128, 129, 130, 131, 132, 133, 134,
                                   135, 136, 137, 138, 139, 140, 141, 142, 143,
                                   144, 145, 146, 147, 148, 149, 150, 151, 152,
                                   153, 154, 155, 156, 157, 158, 159, 160, 161,
                                   162, 163, 164, 165, 166, 167, 168, 169, 170,
                                   171, 172, 173, 174, 175, 176, 177, 178, 179,
                                   180, 181, 182, 183, 184, 185, 186, 187, 188,
                                   189, 190, 191, 192, 193, 194, 195, 196, 197,
                                   198, 199],
                         'colorbar': {'title': {'text': 'Trial'}, 'x': 1.0, 'xpad': 40},
                         'colorscale': [[0.0, 'rgb(247,251,255)'], [0.125,
                                        'rgb(222,235,247)'], [0.25,
                                        'rgb(198,219,239)'], [0.375,
                                        'rgb(158,202,225)'], [0.5,
                                        'rgb(107,174,214)'], [0.625,
                                        'rgb(66,146,198)'], [0.75,
                                        'rgb(33,113,181)'], [0.875,
                                        'rgb(8,81,156)'], [1.0, 'rgb(8,48,107)']],
                         'line': {'color': 'Grey', 'width': 0.5},
                         'showscale': True},
              'mode': 'markers',
              'name': 'Feasible Trial',
              'showlegend': False,
              'type': 'scatter',
              'x': [48, 16, 39, 18, 31, 15, 42, 11, 48, 14, 30, 37, 17, 46, 18,
                    21, 32, 50, 43, 13, 35, 23, 29, 33, 27, 36, 20, 13, 43, 17, 46,
                    27, 30, 23, 38, 20, 11, 21, 30, 41, 35, 44, 34, 19, 25, 14, 37,
                    19, 35, 46, 13, 10, 16, 19, 40, 33, 20, 35, 30, 10, 49, 48, 44,
                    22, 38, 50, 40, 22, 47, 36, 23, 11, 34, 24, 18, 31, 50, 21, 49,
                    17, 39, 40, 20, 14, 38, 29, 35, 35, 16, 34, 36, 25, 12, 38, 10,
                    38, 42, 43, 41, 27, 47, 46, 13, 33, 23, 42, 32, 14, 14, 43, 50,
                    48, 13, 47, 29, 36, 25, 49, 46, 32, 38, 46, 46, 17, 13, 31, 16,
                    39, 45, 43, 25, 29, 45, 32, 20, 34, 24, 41, 32, 11, 18, 34, 24,
                    35, 47, 50, 19, 16, 24, 17, 21, 28, 38, 13, 21, 18, 25, 20, 31,
                    19, 23, 46, 30, 40, 24, 50, 16, 21, 31, 26, 20, 32, 17, 19, 33,
                    12, 50, 37, 26, 49, 17, 44, 16, 31, 43, 22, 46, 35, 23, 31, 22,
                    33, 43, 26, 39, 32, 47, 48, 35, 33],
              'xaxis': 'x',
              'y': [-0.08866610522384985, -0.09265716483159438,
                    -0.0951360731400551, -0.07684681673009439,
                    -0.08383592645315964, -0.08410220744539584,
                    -0.08624530077090894, -0.07952441059356559,
                    -0.09307516554099445, -0.0843572990182119,
                    -0.09400257350281849, -0.08664728232580649,
                    -0

In [87]:
best_n_estimators = best_params['n_estimators']
best_max_depth = best_params['max_depth']
best_min_samples_split = best_params['min_samples_split']
best_min_samples_leaf = best_params['min_samples_leaf']

In [88]:
best_model = RandomForestRegressor(n_estimators=best_n_estimators, max_depth=best_max_depth, min_samples_split=best_min_samples_split, min_samples_leaf=best_min_samples_leaf)
best_model.fit(X_train, y_train)

RandomForestRegressor(max_depth=34, min_samples_leaf=2, n_estimators=358)

In [89]:
y_pred = best_model.predict(X_test)


In [90]:
mean_absolute_error(y_test, y_pred)


0.12329853843438222

In [91]:
mean_squared_error(y_test, y_pred)


0.09388998281868036

In [92]:
r2_score(y_test, y_pred)


0.926003623059204

In [98]:
# Predict on the test set
y_pred = best_model.predict(X_test)


In [99]:
from sklearn.metrics import mean_absolute_percentage_error

# Calculate MAPE
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f"MAPE: {mape * 100}%")


MAPE: 2.2582364567175564e+16%


In [100]:
if np.any(y_test == 0):
    print("Warning: Zero values found in y_test")


In [103]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_elements = y_true != 0
    if np.any(non_zero_elements):
        return np.mean(np.abs((y_true[non_zero_elements] - y_pred[non_zero_elements]) / y_true[non_zero_elements])) * 100
    else:
        return np.inf


# Example: Your actual and predicted values
y_test = np.array([1.1, 2.0, 0.0, 4.1, 5.0])  # Replace with your actual values
y_pred = np.array([1.2, 1.9, 0.1, 4.0, 4.9])  # Replace with your predicted values

# Check for zero values in y_test
if np.any(y_test == 0):
    print("Warning: Zero values found in y_test")

# Calculate MAPE
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f"MAPE: {mape}%")


MAPE: 4.632483370288242%
